# StackOverflow answer classifier

In [5]:
import re
import sys
from datetime import datetime
from html.parser import HTMLParser

import stackexchange
import numpy as np
import pandas as pd
from stackexchange import Site, StackOverflow, Sort, DESC, ASC

import credentials

In [252]:
class HTML2String(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.strict = False
        self.convert_charrefs= True
        self.text = []
    def handle_data(self, d):
        self.text.append(d)
    def get_data(self):
        return ''.join(self.text)

def html_to_string(html):
    s = HTML2String()
    s.feed(html)
    return s.get_data()

##Preparing data

In [59]:
qs_columns = ['id', 'author_id', 'date', 'title', 'text', 'score', 'views', 'tags', 'url']
as_columns = ['id', 'author_id', 'question_id', 'date', 'text', 'code_length', 'score', 'accepted', 'url']
us_columns = ['id', 'name', 'total_questions', 'unanswered_questions', 'total_answers', 'accepted_answers', 'reputation', 'badge_total', 'url']
qa_columns = ['id', 'text', 'q_score', 'a_score', 'qer_reputation', 'aer_reputation', 'qer_percent_answered_questions', 'aer_percent_accepted_answers', 'aer_badge_total', 'q_tag_pop', 'a_code_lenth', 'aer_subscribed_tag', 'is_qa']

questions_df = pd.DataFrame(columns=qs_columns)
answers_df = pd.DataFrame(columns=as_columns)
users_df = pd.DataFrame(columns=us_columns)
qas_df = pd.DataFrame(columns=qa_columns)

In [6]:
so = stackexchange.Site(stackexchange.StackOverflow, app_key=credentials.KEY)
# so.impose_throttling = True
# so.throttle_stop = False
# so.be_inclusive()

In [30]:
from_date = datetime.strptime('2014-01-01', '%Y-%m-%d')
# to_date = datetime.strptime('2014-12-31', '%Y-%m-%d')
to_date = datetime.strptime('2014-01-02', '%Y-%m-%d')

from_date_ts = int(from_date.timestamp())
to_date_ts = int(to_date.timestamp())

In [44]:
questions = so.questions(fromdate=from_date_ts, todate=to_date_ts, sort=Sort.Creation, order=ASC, pagesize=100)

In [112]:
aa=q.answers

In [110]:
ww=so.answers(question_id=questions[6].id)

In [113]:
aa.

AttributeError: 'list' object has no attribute 'total'

In [45]:
questions[0]

<Question 'Chrome crashes when exporting file via Filesystem API' @ 7f1a48b570f0>

In [47]:
questions[0]

<Question 'Chrome crashes when exporting file via Filesystem API' @ 7f1a48b570f0>

In [66]:
df = pd.DataFrame([[1,2,3,4,5,6,7,8, 9]], columns=qs_columns)

In [67]:
questions_df.append(df, ignore_index=True)

,id,author_id,date,title,text,score,views,tags,url
0,1,2,3,4,5,6,7,8,9


In [63]:
questions = so.questions(fromdate=from_date_ts, todate=to_date_ts, sort=Sort.Creation, order=ASC, pagesize=100)
while questions.has_more:
    for question in questions:
        q_id = question.id
        author_id = question.owner_id
        date = question.creation_date
        title = question.title
        text = html_to_string(question.body)
        score = question.score
        views = question.view_count
        tags = question.tags
        url = question.url
        
        info = [q_id, author_id, date, title, text, score, views, tags, url]
        df = pd.DataFrame([info], columns=qs_columns)
        questions_df = questions_df.append(df, ignore_index=True)
        
        answers = question.answers
        for answer in answers:
            
    
    questions = questions.fetch_next()

SyntaxError: invalid syntax (<ipython-input-63-8cd8234eead8>, line 13)

In [469]:
t=so.tag('php')

In [122]:
q.answers

[<Answer 20865938 @ 7f1a4aaea908>]

In [37]:
q.tags

['javascript', 'google-chrome', 'encryption', 'html5-filesystem']

In [462]:
question = so.questions(sort=Sort.Votes, order=DESC)[0]

In [464]:
len(question.comments.fetch())

30

In [451]:
q.owner

<User 'user3148596' (3148596) @ 7f7257202c50>

In [446]:
count=0
for q in questions:
    print(q.title)
    print('-' * 80)
    if count == 10:
        break
    count += 1

Chrome crashes when exporting file via Filesystem API
--------------------------------------------------------------------------------
sort struct array in c++
--------------------------------------------------------------------------------
unsupported operand types for python classes
--------------------------------------------------------------------------------
Play framework "find" (Finder) results missing OneToMany relationship data
--------------------------------------------------------------------------------
Is it possible to set up Amazon EC2 auto-scaling for particular days of the week?
--------------------------------------------------------------------------------
What's the logic of this error?
--------------------------------------------------------------------------------
Multi-line header is not working.
--------------------------------------------------------------------------------
Codeigniter page caching: Is 24Gb of cache files too much?
---------------------------

In [441]:
len(questions)

100

In [447]:
questions.fetch()a

SyntaxError: invalid syntax (<ipython-input-447-bdbe80c41630>, line 1)

In [426]:
questions.total

2349610

In [422]:
questions[29].creation_date

datetime.datetime(2014, 1, 1, 0, 13, 32)

In [381]:
dt = datetime.strptime('1980-06-13', '%Y-%m-%d')

In [391]:
dt.timestamp()

329716800.0

In [393]:
q.creation_date.timestamp()

1340805096.0

In [448]:
qq=so.questions(todate=datetime.strptime('1980-06-13', '%Y-%m-%d').timestamp())a

SyntaxError: invalid syntax (<ipython-input-448-df33aa3b509a>, line 1)

In [354]:
df=pd.DataFrame([1,2,3])

In [357]:
df

,0
0,1
1,2
2,3


In [362]:
df.ix[1]

0    2
Name: 1, dtype: int64

In [366]:
aa=np.array([1,2])

In [368]:
aa.view()

array([1, 2])

In [3]:
me_id = 2055126
user_id = 41981
me = so.user(me_id)
user = so.user(user_id)

In [47]:
rs=user.answers.fetch()

In [48]:
rs.pagesize = 100

In [51]:
len(rs.fetch())

30

In [52]:
questions = so.questions(sort=Sort.Votes, order=DESC)

In [32]:
q=questions[0]

In [59]:
q.owner

<User 'GManNickG' (87234) @ 7f7267d31b38>

In [94]:
q.view_count

595789

In [3]:
answers=so.answers(sort=Sort.Votes, order=DESC)

NameError: name 'so' is not defined

In [2]:
a=answers[0]

NameError: name 'answers' is not defined

In [113]:
a.question_id

11227809

In [260]:
import re

In [270]:
re.findall(r'x.+?x', 'asxyxdasxzxdasda')

['xyx', 'xzx']

In [329]:
re.findall(r'<code>.*?</code>', a.body, re.DOTALL)

['<code>if (data[c] &gt;= 128)\n    sum += data[c];\n</code>',
 '<code>T = branch taken\nN = branch not taken\n\ndata[] = 0, 1, 2, 3, 4, ... 126, 127, 128, 129, 130, ... 250, 251, 252, ...\nbranch = N  N  N  N  N  ...   N    N    T    T    T  ...   T    T    T  ...\n\n       = NNNNNNNNNNNN ... NNNNNNNTTTTTTTTT ... TTTTTTTTTT  (easy to predict)\n</code>',
 '<code>data[] = 226, 185, 125, 158, 198, 144, 217, 79, 202, 118,  14, 150, 177, 182, 133, ...\nbranch =   T,   T,   N,   T,   T,   T,   T,  N,   T,   N,   N,   T,   T,   T,   N  ...\n\n       = TTNTTTTNTNNTTTN ...   (completely random - hard to predict)\n</code>',
 '<code>if (data[c] &gt;= 128)\n    sum += data[c];\n</code>',
 '<code>int t = (data[c] - 128) &gt;&gt; 31;\nsum += ~t &amp; data[c];\n</code>',
 '<code>data[]</code>',
 '<code>//  Branch - Random\nseconds = 11.777\n\n//  Branch - Sorted\nseconds = 2.352\n\n//  Branchless - Random\nseconds = 2.564\n\n//  Branchless - Sorted\nseconds = 2.587\n</code>',
 '<code>//  Branch - Ra

In [316]:
qqq='<pre><code>if (data[c] &gt;= 128)\n    sum += data[c];\n</code></pre>'

In [317]:
qqq

'<pre><code>if (data[c] &gt;= 128)\n    sum += data[c];\n</code></pre>'

In [325]:
'1  4  7  0'.split(' ')

['1', '', '4', '', '7', '', '0']

In [343]:
l=['asdasd', '', 'utyutyu']
l = filter(lambda x: len(x) > 0, l)

In [344]:
for x in l:
    print(x)

asdasd
utyutyu


In [347]:
codes = re.findall(r'<code>.*?</code>', a.body, re.DOTALL)
count_lines = 0
for code in codes:
    text = html_to_string(code)
    lines = text.split('\n')
    lines = list(filter(lambda x: len(x) > 0, lines))
    count_lines += len(lines)
print(count_lines)

34


In [323]:
a.body[:100]

'<p><strong>You are the victim of <a href="//en.wikipedia.org/wiki/Branch_predictor">branch predictio'

In [254]:
html_to_string(a.body)[:100]

'You are the victim of branch prediction fail.\n\n\n\nWhat is Branch Prediction?\n\nConsider a railroad jun'

In [7]:
users=so.users()

In [8]:
u=users[0]

In [16]:
so.impose_throttling = False
so.throttle_stop = True

In [25]:
u.answers.

(<stackexchange.site.Site at 0x7f1a68763320>,
 'users/22656/answers',
 stackexchange.models.Answer,
 'answers',
 {'body': 'true', 'comments': 'true', 'site': 'stackoverflow.com'})

In [23]:
aa=so.answers(user_id=u.id)

In [34]:
cc=so.comments(question_id=q.id)

In [36]:
cc.

(<stackexchange.models.Comment at 0x7f1a682cccf8>,
 ...)

In [26]:
aa.total

31668

In [237]:
tags=so.tags()

In [242]:
t=tags[0]

In [248]:
t.

109957

In [232]:
type(questions)

stackexchange.core.StackExchangeResultset

####Experience

In [4]:
print('StackOverflow user %d\'s experience:' % user_id)

print('Most experienced on %s.' % user.top_answer_tags.fetch()[0].tag_name)
print('Most curious about %s.' % user.top_question_tags.fetch()[0].tag_name)

total_questions = len(user.questions.fetch())
unaccepted_questions = len(user.unaccepted_questions.fetch())
accepted = total_questions - unaccepted_questions
rate = accepted / float(total_questions) * 100
print('Accept rate is %.2f%%.' % rate)

StackOverflow user 41981's experience:
Most experienced on css.
Most curious about .net.
Accept rate is 84.62%.


####Highest voted

In [5]:
print('The highest voted question on StackOverflow is:')
question = so.questions(sort=Sort.Votes, order=DESC)[0]
print('\t%8d\t%s\t%d' % (question.id, question.title, question.score))
print()
print('Look, see:', question.url)

The highest voted question on StackOverflow is:
	11227809	Why is processing a sorted array faster than an unsorted array?	10693

Look, see: http://stackoverflow.com/questions/11227809


####Question

In [6]:
#id = int(get_input("Enter a question ID: "))
id = 11227809
question = so.question(id)

print('--- %s ---' % question.title)
print(question.body)
print()
print('%d answers.' % len(question.answers))

--- Why is processing a sorted array faster than an unsorted array? ---
<p>Here is a piece of <strong>C++</strong> code that seems very peculiar. For some strange reason, sorting the data miraculously makes the code almost six times faster.</p>

<pre class="lang-cpp prettyprint-override"><code>#include &lt;algorithm&gt;
#include &lt;ctime&gt;
#include &lt;iostream&gt;

int main()
{
    // Generate data
    const unsigned arraySize = 32768;
    int data[arraySize];

    for (unsigned c = 0; c &lt; arraySize; ++c)
        data[c] = std::rand() % 256;

    // !!! With this, the next loop runs faster
    std::sort(data, data + arraySize);

    // Test
    clock_t start = clock();
    long long sum = 0;

    for (unsigned i = 0; i &lt; 100000; ++i)
    {
        // Primary loop
        for (unsigned c = 0; c &lt; arraySize; ++c)
        {
            if (data[c] &gt;= 128)
                sum += data[c];
        }
    }

    double elapsedTime = static_cast&lt;double&gt;(clock() - start) / 

####Recent questions

In [7]:
sys.stdout.write('Loading...')
sys.stdout.flush()

questions = so.recent_questions(pagesize=10, filter='_b')
print('\r #  vote ans view')

cur = 1
for question in questions[:10]:
    print('%2d %3d  %3d  %3d \t%s' % (cur, question.score, len(question.answers), question.view_count, question.title))
    cur += 1

#num = int(get_input('Question no.: '))
num = 1
qu  = questions[num - 1]
print()
print('--- TITLE: %s' % qu.title)
print('%d votes, %d answers, %d views.' % (qu.score, len(qu.answers), qu.view_count))
print('Tagged: ' + ', '.join(qu.tags))
print()
print(qu.body[:250] + ('...' if len(qu.body) > 250 else ''))

 #  vote ans view
 1  11    0  9633 	System versus user PATH environmental variable...winmerge works only if I add the path to the user PATH
 2   0    0    4 	Referencing unbounded function in Clojure REPL
 3   0    0    2 	Make bootstrap column touch the bottom of the div
 4   0    0    2 	Select to Return Multiple Rows from Single Row
 5   0    0    7 	What is the best way to incrementally build a numpy array?
 6   0    0   12 	Why is my sanitized img tags coming up empty?
 7   1    0  597 	Highcharts - Force plot lines to consume available width?
 8   0    0    2 	Strange behaviour when I trying to override protected static variable of Abstract class
 9   0    0    8 	Requesting data from Form using Jquery - have to refresh to resubmit
10   0    0   30 	Upload images PHP to MySQL and retrieve correct images to each form

--- TITLE: System versus user PATH environmental variable...winmerge works only if I add the path to the user PATH
11 votes, 0 answers, 9633 views.
Tagged: windows,

####Search

In [8]:
term = 'python'
print('Searching for %s...' % term,)
sys.stdout.flush()

qs = so.search(intitle=term)

print('\r--- questions with "%s" in title ---' % (term))

for q in qs[:10]:
    print('%8d %s' % (q.id, q.title))

Searching for python...
--- questions with "python" in title ---
30468555 List index out of range error (Python 2.7)
30468977 Why does Python String concatination work with Russian text but string.format() dose not
30469120 Python Django migrate_schemas --shared TypeError: hasattr(): attribute name must be string upon
30426935 Reassign rows and columns in Python csv
30468920 Python-linkedin Return URL?
30468371 How to convert python timestamp string to epoch?
30468593 Adding Buffer Protocol to Python Class (Not at C Level)
30468461 Finding combinations in a list (Python)
30467495 mysql installer fails to recognize python 3.4
30468902 How can I control access to a Wiifi Router using Python-Flask?


####Stats

In [9]:
stats = so.stats()

print('Total questions:\t%d' % stats.total_questions)
print('\tAnswered:\t%d' % (stats.total_questions - stats.total_unanswered))
print('\tUnanswered:\t%d' % (stats.total_unanswered))

percent = (stats.total_unanswered / float(stats.total_questions)) * 100
print('%.2f%% unanswered. (%.2f%% answered!)' % (percent, 100 - percent))

Total questions:	9470823
	Answered:	7000654
	Unanswered:	2470169
26.08% unanswered. (73.92% answered!)
